In [1]:
from __future__ import print_function
import tweepy
import json
from pymongo import MongoClient
import numpy as np
import pandas as pd
from datetime import datetime
import time
from datetime import timedelta
from kafka import KafkaProducer
import requests

In [2]:
def troncahashtag (lista):
    for i in range(0, len(lista)):
        lista[i] = lista[i][1:]
    return lista

In [4]:
CONSUMER_KEY = "key" 
CONSUMER_SECRET = "key"

ACCESS_TOKEN = "key" 
ACCESS_TOKEN_SECRET = "key" 

In [5]:
def minuscolo(hashtags):
    for i in range(0,len(hashtags)):
        hashtags[i] = hashtags[i].lower()
    return (hashtags)

In [6]:
now=datetime.now()
Match = minuscolo(["#WeTheNorth", "#Raptors", "#TorontoRaptors", "#NBAFinals", "#NBAPlayoffs", "RTZ",
                   "#DubNation", "#WarriorsGround", "#StrengthInNumbers", "#GoldenStateWarriors", "#GswvsTor", "#TorvsGsw" ])
Away = minuscolo(["#WeTheNorth", "#Raptors", "#TorontoRaptors", "RTZ"])
Home = minuscolo(["#DubNation", "#WarriorsGround", "#StrengthInNumbers", "#GoldenStateWarriors"])
N = minuscolo(["#NBAFinals", "#NBAPlayoffs", "#GswvsTor", "#TorvsGsw"])


In [7]:
producer = KafkaProducer(bootstrap_servers='sandbox-hdp.hortonworks.com:6667',
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

## Stats

In [8]:
def integrazione(tweetFiltered, statistiche):
    tweetFiltered["stats"] = statistiche
    return tweetFiltered

## Listener

In [13]:
i = 407003

In [14]:
class StreamListener(tweepy.StreamListener):    
 
    def on_connect(self):
        print("Connessione con l'API avvenuta")
 
    def on_error(self, status_code):
        print('An Error has occured: ' + repr(status_code))
        return False
 
    def on_data(self, data):
        
        try:
                %store -r statistiche    # con questi 2 comandi carico su questo notebook le variabili statistiche e richiesta
                %store -r richiesta      # calcolate in un altro notebook
                statistiche
                richiesta
                global i
                datajson = json.loads(data) #datajson contiene tutta la struttura del tweet
                
                timeTweet=datajson['created_at']
                userLoc=datajson['user'].get('location')
                tweetFiltered={'_id': datajson['id_str'],
                               'n_tweet' : i,
                               'time_tweet': time.strftime("%H:%M:%S"),
                               'date_tweet' : time.strftime("%Y-%m-%d"),
                               'source': datajson['source'],
                               'retweet_count':datajson['retweet_count'],
                               'placeTweet': datajson['coordinates'],
                               'placeUser': userLoc}
                tweetFiltered = integrazione(tweetFiltered,statistiche)
                i = i+1
                
                if ('retweeted_status' in datajson): #controllo se oggetto è retweet no commentato oppure tweet/retweet commentato
                        if ('extended_tweet' in datajson['retweeted_status']): #controllo se retweet non commentato è lungo oppure corto
                            text=datajson['retweeted_status']['extended_tweet']['full_text']
                            print("RETWEET NON COMMENTATO LUNGO:", text)
                            hashtag=minuscolo(flatted(datajson['retweeted_status']['extended_tweet']['entities']['hashtags']))
                        else:
                            text=datajson['retweeted_status']['text']
                            print("RETWEET NON COMMENTATO CORTO: ", text)
                            hashtag=minuscolo(flatted(datajson['retweeted_status']['entities']['hashtags']))
                        idRetweet=datajson['retweeted_status']['id_str']
                        tweetFiltered['retweetId']=idRetweet
                else:
                    if (datajson['is_quote_status']== True): #controllo se retweeet è commentato o no
                        print("COMMENTO")
                        idQuote=datajson['quoted_status_id_str']
                        if ('extended_tweet' in datajson): #controllo se commento è lungo oppure corto
                            if ('extended_tweet' in datajson['quoted_status']): #controllo se original lungo o corto
                                text=datajson['extended_tweet']['full_text']
                                hashtagQuote=flatted(datajson['extended_tweet']['entities']['hashtags'])
                                hashtagOriginal=flatted(datajson['quoted_status']['extended_tweet']['entities']['hashtags'])
                                hashtag=minuscolo(hashtagQuote+hashtagOriginal)
                                print("RETWEET COMMENTATO LUNGO ORIGINALE LUNGO: ", text)
                            else:
                                text=datajson['extended_tweet']['full_text']
                                hashtagQuote=flatted(datajson['extended_tweet']['entities']['hashtags'])
                                hashtagOriginal=flatted(datajson['quoted_status']['entities']['hashtags'])
                                hashtag=minuscolo(hashtagQuote+hashtagOriginal)
                                print("RETWEET COMMENTATO LUNGO ORIGINALE CORTO: ", text)
                        else:
                            if ('extended_tweet' in datajson['quoted_status']): #controllo se original lungo o corto
                                text=datajson['text']
                                hashtagQuote=flatted(datajson['entities']['hashtags'])
                                hashtagOriginal=flatted(datajson['quoted_status']['extended_tweet']['entities']['hashtags'])
                                hashtag=minuscolo(hashtagQuote+hashtagOriginal)
                                print("RETWEET COMMENTATO CORTO ORIGINALE LUNGO: ", text)
                            else:
                                text=datajson['text']
                                hashtagQuote=flatted(datajson['entities']['hashtags'])
                                hashtagOriginal=flatted(datajson['quoted_status']['entities']['hashtags'])
                                hashtag=minuscolo(hashtagQuote+hashtagOriginal)
                                print("RETWEET COMMENTATO CORTO ORIGINALE CORTO: ", text)
                        tweetFiltered['retweetId']=idQuote
                    else:
                        if ('extended_tweet' in datajson):
                            text=datajson['extended_tweet']['full_text']
                            print("TWEET LUNGO: ", text)
                            hashtag=minuscolo(flatted(datajson['extended_tweet']['entities']['hashtags']))
                        else:
                            text=datajson['text']
                            print("TWEET BREVE: ", text)
                            hashtag=minuscolo(flatted(datajson['entities']['hashtags']))
                tweetFiltered['text']=text
                tweetFiltered['hashtag']=list(set(hashtag))
                print(" Gli hashtag sono: ", hashtag)
                print("\n\n")   
                        
                #print("NOW: %s | ORA LIMITE: %s" % (now,prova))
                
                ########################################
                #QUI INSERISCO IL CODICE PER INVIO SUL TOPIC
                producer.send('gara1', [tweetFiltered, richiesta, data])
                
                
                ########################################
                
                tweetFiltered.clear()
         
        except Exception as e:
            print(e)

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)


listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)


ora_now=datetime.now()
ora_partita=datetime(ora_now.year,ora_now.month,ora_now.day+1,1,0) #orario GMT (2 ore indietro italia)
inizio_stream=ora_partita-timedelta(hours=1)
print("ORA PARTITA: %s | ORA INIZIO STREAM: %s " %(ora_partita,inizio_stream))

print("NOW: %s | CONFRONTO: %s\n" %(ora_now,inizio_stream))

while (datetime.now()<inizio_stream):
    print("WAIT: %s" % datetime.now())
    time.sleep(5)


print("INIZIO STREAM: %s\n" % datetime.now())
print("Sto ascoltando: " + str(Match))
streamer.filter(track=Match,languages=['en'])

ORA PARTITA: 2019-06-06 01:00:00 | ORA INIZIO STREAM: 2019-06-06 00:00:00 
NOW: 2019-06-05 17:16:38.178674 | CONFRONTO: 2019-06-06 00:00:00

WAIT: 2019-06-05 17:16:38.179099
WAIT: 2019-06-05 17:16:43.183034
